In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [2]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

In [3]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])


model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#model.save('mnist.h5')
#print("Saving the model as mnist.h5")

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/10
469/469 [==============================] - 111s 235ms/step - loss: 2.2872 - accuracy: 0.1346 - val_loss: 2.2513 - val_accuracy: 0.2897
Epoch 2/10
469/469 [==============================] - 120s 256ms/step - loss: 2.2273 - accuracy: 0.2642 - val_loss: 2.1774 - val_accuracy: 0.5319
Epoch 3/10
469/469 [==============================] - 107s 229ms/step - loss: 2.1492 - accuracy: 0.3870 - val_loss: 2.0764 - val_accuracy: 0.6520
Epoch 4/10
469/469 [==============================] - 109s 233ms/step - loss: 2.0372 - accuracy: 0.4872 - val_loss: 1.9316 - val_accuracy: 0.7124
Epoch 5/10
469/469 [==============================] - 110s 235ms/step - loss: 1.8784 - accuracy: 0.5628 - val_loss: 1.7253 - val_accuracy: 0.7461
Epoch 6/10
469/469 [==============================] - 109s 234ms/step - loss: 1.6687 - accuracy: 0.6129 - val_loss: 1.4660 - val_accuracy: 0.7718
Epoch 7/10
469/469 [==============================]

In [4]:
#from keras.models import load_model
from tkinter import *
import tkinter as tk

import win32gui
from PIL import ImageGrab, Image
import numpy as np
#model = load_model('mnist.h5')


# In[2]:


def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    #res = model.predict([img])[0]
    res = model.predict([img])[0]
    print(res)
    return np.argmax(res), max(res)


# In[ ]:


class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)
    def clear_all(self):
        self.canvas.delete("all")
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')
    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
app = App()
mainloop()

1/1 [==============================] - 0s 142ms/step
[2.4621119e-01 8.7288186e-05 2.7042606e-01 8.8689923e-02 1.6075301e-03
 3.5216063e-01 9.3611109e-04 1.7080666e-03 3.7946004e-02 2.2714429e-04]
1/1 [==============================] - 0s 29ms/step
[2.3923253e-01 4.8720627e-05 1.9242114e-01 1.6970800e-01 1.7846649e-03
 3.7271068e-01 1.7487158e-03 1.3052443e-03 2.0882061e-02 1.5829026e-04]
1/1 [==============================] - 0s 37ms/step
[3.1724066e-01 5.6839610e-05 1.7339535e-01 1.2070605e-01 2.7477285e-03
 3.5620990e-01 1.4049122e-03 2.2803890e-03 2.5664575e-02 2.9353955e-04]
1/1 [==============================] - 0s 34ms/step
[2.60488480e-01 1.04121427e-04 2.09666193e-01 1.06516175e-01
 2.86209863e-03 3.69752616e-01 1.28552516e-03 3.60870361e-03
 4.53340597e-02 3.82083992e-04]
1/1 [==============================] - 0s 27ms/step
[3.5598511e-01 4.7253641e-05 1.4266719e-01 1.4388202e-01 2.1120079e-03
 3.2935014e-01 1.6214460e-03 1.9316623e-03 2.2170177e-02 2.3299007e-04]
1/1 [========